# Big Data Project

## Configuration

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2122-dgrushchak"

val path = "s3a://"+bucketname+"/project/small_file_10000000.txt" 

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1674204611665_0007,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-dgrushchak
path: String = s3a://unibo-bd2122-dgrushchak/project/small_file_10000000.txt
res3: String = application_1674204611665_0007
res5: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1674204611665_0007/


In [5]:
// import java.util.Calendar
// import org.apache.spark.sql.SaveMode
// import org.apache.spark.HashPartitioner
val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
// object Parser {

//       val noGenresListed = "(no genres listed)"
      
//       val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
//       val quotes = "\""

//       //TODO check
//       /** Convert from timestamp (String) to year (Int) */
//       def yearFromTimestamp(timestamp: String): Int = {
//         val cal = Calendar.getInstance()
//         cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
//         cal.get(Calendar.YEAR)
//       }


//         // TODO redo
//       /** Function to parse rating records
//        *
//        *  @param line line that has to be parsed
//        *  @return tuple containing userId, movieId, rating, and year none in case of input errors
//        */
//       def parseRatingLine(line: String): Option[(Long, Long, Double, Int)] = {
//         try {
//           val input = line.split(commaRegex)
//           Some(input(0).trim.toLong, input(1).trim.toLong, input(2).trim.toDouble, yearFromTimestamp(input(3)))
//         } catch {
//           case _: Exception => None
//         }
//       }
    

//     /**
//     2 - flightdate
//     3 - starting airport
//     4 - destination airport
//     6 - travel duration (PT2H35M)
//     8 - isBasicEconomy (boolean)
//     9 - isrefundable (boolean)
//     10 - isNonStop (boolean)
//     11 - basefare (prezzo base del ticket in $ Double)
//     12 - seats remaining (Int)
//     20 - airline name (company name)
//     22 - airplane type (Vector of [String||String]
//     24 - segment distancd (distance in milles: Vector of [Int||Int]  can be [None||None])
//     **/
   
// }

case class FlightData(
    startAirport:String,
    destinationAirport:String
)

object FlightData {
     def extract(line: String) = {
//         val input = line.split(commaRegex)
        new FlightData("one", "two")
    }
}
    


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commaRegex: String = ,(?=(?:[^"]*"[^"]*")*[^"]*$)
defined class FlightData
defined object FlightData
Companions must be defined together; you may wish to use :paste mode for this.


In [16]:
val rdd = sc.textFile(path).map(FlightData.extract)
rdd.take(10)
println("cio")
// rdd.limit(10).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted due to stage failure: ClassNotFound with classloader: org.apache.livy.rsc.driver.MutableClassLoader@408a0c6d
  at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2863)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2799)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2798)
  at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
  at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
  at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
  at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2798)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1239)
  at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1239)
  at scala

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights flights


In [37]:
rdd.coalesce(12).partitioner
val rddCache = rdd.cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res20: Option[org.apache.spark.Partitioner] = None
rddCache: rdd.type = MapPartitionsRDD[20] at flatMap at <console>:40


In [41]:
val ris = rddCache.map(x => x._1).concat(rdd.cache.map(x => x._2)).distinct.count
ris

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:39: error: value concat is not a member of org.apache.spark.rdd.RDD[String]
       val ris = rddCache.map(x => x._1).concat(rdd.cache.map(x => x._2)).distinct.count
                                         ^



## Personal works

> Basta una query compicata a testa

Query approfondite:

- Denys:
    - Aggrego “airline” e “aircraft model” per determinare il prezzo medio di un miglio nautico, e dopo aggrego (MIN) su “aircraft model” per trovare l'airline che vende i biglietti più economici per quel modello.
    - Aggrego su “aircraft model” per calcolare la “travel distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.
- Riccardo:
    - Aggrego su “aircraft models” per calcolare la classifica discreta dei modelli più usati rispetto a ciascuna “airline”. Eseguo il join col dataset originale, infine riaggrego su “travel duration” e sulla classificazione di prima. Ottengo la durata di media di ogni volo per ogni compagnia (oltre a numero di voli e totale di ore) sul modello di aereo più utilizzato
    - Aggrego su giorno ed airline per calcolare la distanza totale, poi aggrego su airline per calcolare la media dei totali giornalieri (di distanza) per ciascuna compagnia aerea
    - Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni copia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo